In [4]:
import random
import matplotlib.pyplot as plt
import numpy as np
import pickle
%run Sub_Class.ipynb
s= Problem_MIP(number_of_order=800, number_of_sku=2148, number_of_unique_sku=1503, number_of_batch=50)
s.form_data()
#s.To_Original()
%run Obj_Matrix_Return.ipynb
%run Obj_Matrix_Sshape.ipynb


In [40]:
import pickle
 
with open('s1.pkl', 'wb') as config_dictionary_file:
     pickle.dump(s1, config_dictionary_file)

In [56]:
file=open("s1.pkl","rb")
s=pickle.load(file)

In [9]:
Origin_list=[i for i in range(s.number_of_order)]
def Corr_Analysis(s, matrix):
    #dis = len(s.mb) * s.number_of_batch
    dis = 0
    cat = 0
    for batch in matrix:
        set_of_shelf = set()
        set_of_sku = []
        for i in batch:
            for j in s.Ki[i]:
                location = s.skubin[s.list_of_unique_sku[j]]
                set_of_shelf.add(location[0])
            set_of_sku.extend(set(s.Ki[i]))
        dis += np.sum([s.mb[int(i) - 1] for i in set_of_shelf])
        #dis+= len(set_of_shelf)

        # Now we assum that all shelf is length 1
        #dis += len(set_of_shelf)
        cat += len(set(set_of_sku))
    return cat, dis

In [10]:
result=[[],[],[],[]]
n=5000 # Number of samples

for i in range(n):
    random.shuffle(Origin_list)
    matrix=[[] for i in range(s.number_of_batch)]
    perunit=int(s.number_of_order/s.number_of_batch)
    for j in range(s.number_of_batch):
        matrix[j].extend(Origin_list[j*perunit:(j+1)*perunit])
    cat,dis=Corr_Analysis(s,matrix)
    result[0].append(cat)
    result[1].append(dis)
    result[2].append(Obj_Matrix_Sshape_solver(s, matrix))
    result[3].append(Obj_Matrix_Return_solver(s, matrix))

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
c={'cat' : result[0],'dis' : result[1],'S':result[2],'Return':result[3]}
data= pd.DataFrame(c)


In [8]:
print(data.corr())

             cat       dis         S    Return
cat     1.000000  0.159970  0.144367  0.209755
dis     0.159970  1.000000  0.846964  0.714366
S       0.144367  0.846964  1.000000  0.692896
Return  0.209755  0.714366  0.692896  1.000000


In [12]:
print(data.corr())

             cat       dis         S    Return
cat     1.000000  0.125436  0.122401  0.200469
dis     0.125436  1.000000  0.973157  0.726792
S       0.122401  0.973157  1.000000  0.685214
Return  0.200469  0.726792  0.685214  1.000000


In [25]:
#data.to_excel('corr_exp.xlsx')

In [26]:
re=data.to_numpy()
np.save('result.npy',re)

In [73]:
w=25
plt.figure()
plt.hist(data['S'], bins=np.arange(min(data['S']), max(data['S']) + w, w),density=True,label="Histogram of samples") 
#plt.show()



from scipy.stats import norm
import statistics
  
# Plot between -10 and 10 with .001 steps.
x_axis = np.arange(min(data['S']), max(data['S']), 1)
  
# Calculating mean and standard deviation
mean = statistics.mean(x_axis)
sd = statistics.stdev(x_axis)
  
#plt.plot(x_axis, norm.pdf(x_axis, mean, sd))
plt.plot(x_axis, norm.pdf(x_axis, data['S'].mean(), data['S'].std()),label='PDF')
plt.legend()
#plt.show()
plt.savefig("Sshape.pdf")


In [74]:
from scipy import stats
result = stats.kstest(data['S'], 'norm',(data['S'].mean(),data['S'].std()))
print(result)

KstestResult(statistic=0.009585309823807231, pvalue=0.7440186495303216)


In [75]:
w=25
plt.figure()
plt.hist(data['Return'], bins=np.arange(min(data['Return']), max(data['Return']) + w, w),density=True,label="Histogram of samples")  
#plt.show()
plt.savefig("Return.eps",format="eps")

# Plot between -10 and 10 with .001 steps.
x_axis = np.arange(min(data['Return']), max(data['Return']), 1)
  
# Calculating mean and standard deviation
mean = statistics.mean(x_axis)
sd = statistics.stdev(x_axis)
  
#plt.plot(x_axis, norm.pdf(x_axis, mean, sd))
plt.plot(x_axis, norm.pdf(x_axis, data['Return'].mean(), data['Return'].std()),label='PDF')
plt.legend()
#plt.show()
plt.savefig("Return.pdf")


In [76]:
from scipy import stats
result = stats.kstest(data['Return'], 'norm',(data['Return'].mean(),data['Return'].std()))
print(result)

KstestResult(statistic=0.009193660659844671, pvalue=0.7882423936685541)


In [65]:
from scipy import stats
shapiro_test = stats.shapiro(data['dis'])
shapiro_test

ShapiroResult(statistic=0.9991068243980408, pvalue=0.010199903510510921)

In [246]:
import numpy as np
import statsmodels.api as sm
import pylab as py

# np.random generates different random numbers
# whenever the code is executed
# Note: When you execute the same code
# the graph look different than shown below.

# Random data points generated

 
sm.qqplot((data['dis']-data['dis'].mean())/data['dis'].std(), line ='45')
py.show()


C:\Users\MSI-NB\anaconda3\lib\site-packages\statsmodels\graphics\gofplots.py:993: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "bo" (-> marker='o'). The keyword argument will take precedence.
  ax.plot(x, y, fmt, **plot_style)


In [258]:
from statsmodels.graphics.gofplots import qqplot_2samples
x = (data['dis']-data['dis'].mean())/data['dis'].std()
y = (data['S']-data['S'].mean())/data['S'].std()
#y = np.random.uniform(0,1,size=len(x))
pp_x = sm.ProbPlot(x)
pp_y = sm.ProbPlot(y)
qqplot_2samples(pp_x, pp_y)
plt.show()

C:\Users\MSI-NB\anaconda3\lib\site-packages\statsmodels\graphics\gofplots.py:993: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "bo" (-> marker='o'). The keyword argument will take precedence.
  ax.plot(x, y, fmt, **plot_style)


In [245]:
w=2
plt.figure()
plt.hist(data['dis'], bins=np.arange(min(data['dis']), max(data['dis']) + w, w),density=True,label="Histogram of samples") 
x_axis = np.arange(min(data['dis']), max(data['dis']), 1)

plt.plot(x_axis, norm.pdf(x_axis, data['dis'].mean(), data['dis'].std()),label='PDF')

plt.show()

In [66]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of *x* and *y*.

    Parameters
    ----------
    x, y : array-like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    **kwargs
        Forwarded to `~matplotlib.patches.Ellipse`

    Returns
    -------
    matplotlib.patches.Ellipse
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=facecolor, **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)
def get_correlated_dataset(n, dependency, mu, scale):
    latent = np.random.randn(n, 2)
    dependent = latent.dot(dependency)
    scaled = dependent * scale
    scaled_with_offset = scaled + mu
    # return x and y of the new, correlated dataset
    return scaled_with_offset[:, 0], scaled_with_offset[:, 1]
np.random.seed(0)

PARAMETERS = {
    'Positive correlation': [[0.85, 0.35],
                             [0.35, 0.65]],
    'Negative correlation': [[0.9, -0.4],
                             [0.1, -0.6]],
    'Weak correlation': [[1, 0],
                         [0, 1]],
}

mu = 2, 4
scale = 5, 5

fig, axs = plt.subplots(1, 3, figsize=(9, 3))
for ax, (title, dependency) in zip(axs, PARAMETERS.items()):
    x, y = get_correlated_dataset(800, dependency, mu, scale)
    #ax.scatter(x, y, s=0.5)

    ax.axvline(c='grey', lw=1)
    ax.axhline(c='grey', lw=1)

    confidence_ellipse(x, y, ax, edgecolor='red')

    ax.scatter(mu[0], mu[1], c='red', s=3)
    ax.set_title(title)

plt.show()

In [77]:
import seaborn as sns
#ax_nstd.legend()
plot=sns.jointplot(np.array(data['S']),np.array(data['dis']),kind="reg",scatter_kws={'s': 5},line_kws={'color':'black'})
#confidence_ellipse(np.array(data['S']), np.array(data['dis']), ax=plot.ax_joint, n_std=3,label=r'$3\sigma$', edgecolor='firebrick',lw=2)
confidence_ellipse(np.array(data['S']), np.array(data['dis']), ax=plot.ax_joint, n_std=1,label=r'$1\sigma$', edgecolor='firebrick',lw=2)
confidence_ellipse(np.array(data['S']), np.array(data['dis']), ax=plot.ax_joint, n_std=2,label=r'$2\sigma$', edgecolor='fuchsia', lw=1.5)#,linestyle='--')
confidence_ellipse(np.array(data['S']), np.array(data['dis']), ax=plot.ax_joint, n_std=3,label=r'$3\sigma$', edgecolor='blue', lw=1.5)#,linestyle=':')

plot.ax_joint.scatter(data['S'].mean(), data['dis'].mean(), c='red', s=3)
plot.ax_joint.set_xlabel('$X ^S _n$', fontweight='bold',fontsize=16)
plot.ax_joint.set_ylabel('$Y ^\pi _n$', fontweight='bold',fontsize=16)
plot.fig.set_figwidth(12)
plot.fig.set_figheight(10)
#plt.legend()
plot.ax_joint.legend()
#plt.show()
plt.savefig("Y_1S.pdf")


C:\Users\MSI-NB\anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


In [78]:
plot=sns.jointplot(np.array(data['Return']),np.array(data['dis']),kind="reg",scatter_kws={'s': 5},line_kws={'color':'black'})
#confidence_ellipse(np.array(data['Return']), np.array(data['dis']), ax=plot.ax_joint, n_std=3,label=r'$3\sigma$', edgecolor='firebrick',lw=2)
confidence_ellipse(np.array(data['Return']), np.array(data['dis']), ax=plot.ax_joint, n_std=1,label=r'$1\sigma$', edgecolor='firebrick',lw=2)
confidence_ellipse(np.array(data['Return']), np.array(data['dis']), ax=plot.ax_joint, n_std=2,label=r'$2\sigma$', edgecolor='fuchsia', lw=1.5)#,linestyle='--')
confidence_ellipse(np.array(data['Return']), np.array(data['dis']), ax=plot.ax_joint, n_std=3,label=r'$3\sigma$', edgecolor='blue', lw=1.5)#,linestyle=':')

plot.ax_joint.scatter(data['Return'].mean(), data['dis'].mean(), c='red', s=3)
plot.ax_joint.set_xlabel('$X ^R _n$', fontweight='bold',fontsize=16)
plot.ax_joint.set_ylabel('$Y ^\pi _n$', fontweight='bold',fontsize=16)
plot.fig.set_figwidth(12)
plot.fig.set_figheight(10)
#plt.legend()
plot.ax_joint.legend()
#plt.show()
plt.savefig("Y_1R.pdf")

C:\Users\MSI-NB\anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(
